In [1]:
#Import the first set of libraries
import os
import cv2
import numpy as np
import scipy as sp
import getData as gD
import unetkeras as uk
from keras.models import Sequential 
from keras.layers import (Conv2D,MaxPooling2D,Activation,Flatten,Dense,Dropout)
from keras.optimizers import Adam

Using TensorFlow backend.


In [ ]:
trainDir = '/Users/computer/Documents/xRayDataset/trainingSet/'
testDir = '/Users/computer/Documents/xRayDataset/testingSet/'
x_train,Y1,Y2 = gD.trainingData(trainDir,512) #Y2 is the binary labels and Y1 are the masks/segmented images.
x_test,y_test = gD.getTestData(testDir,512)
print(x_train.shape)
print(Y1.shape)
print(Y2.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
#Build a simple Neural Network and train it the regular way
model = Sequential()
model.add(Conv2D(256,kernel_size = 3,activation = 'relu',strides = 2,input_shape = (512,512,1),padding = 'same'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,kernel_size = 3,activation = 'relu',strides = 2,padding = 'same'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,kernel_size = 3,activation = 'relu',strides = 2,padding = 'same'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'rmsprop',metrics = ['accuracy'])
model.summary()

In [4]:
#Fit the model in a direct fit
nEpochs = 50
history = model.fit(x_train,Y2,epochs = nEpochs,validation_split = 0.2,batch_size = 32,verbose = 2)
lossDirect = history.history['loss']
#x_test = gD.getTestData(testDir,newSize)
#preds = model.predict(x_test)


Train on 563 samples, validate on 141 samples
Epoch 1/10
 - 242s - loss: 0.7481 - accuracy: 0.5471 - val_loss: 1.6639 - val_accuracy: 0.4326
Epoch 2/10
 - 234s - loss: 0.6523 - accuracy: 0.7389 - val_loss: 0.7517 - val_accuracy: 0.5035
Epoch 3/10


E0530 21:49:41.952898 4545959360 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-90a28488315e>", line 2, in <module>
    history = model.fit(images,labels,epochs = 10,validation_split = 0.2,batch_size = 32,verbose = 2)
  File "/usr/local/lib/python3.7/site-packages/keras/engine/training.py", line 1239, in fit
    validation_freq=validation_freq)
  File "/usr/local/lib/python3.7/site-packages/keras/engine/training_arrays.py", line 196, in fit_loop
    outs = fit_function(ins_batch)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/keras/backend.py", line 3792, in __call__
    outputs = self._graph_fn(*converted_inputs)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/eager/function.py", line 1605, in __call__
    return self._call_impl(args, kwargs)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/eager/f

KeyboardInterrupt: 

Next, let's employ K-fold cross-validation since the number of images we have is less, just 704 training images
and 96 test images. The model used in k-fold is the same one as in direct training before moving on to Unet image 
segmentation and creating masks. The targets or training labels are the masks given and the testing set can remain the same 

In [ ]:
def getModel(inShape):
    #inShape is the input shape of the image
    #Define a function to get the model
    model = Sequential()
    model.add(Conv2D(256,kernel_size = 3,activation = 'relu',strides = 2,input_shape = inShape,padding = 'same'))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(128,kernel_size = 3,activation = 'relu',strides = 2,padding = 'same'))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(128,kernel_size = 3,activation = 'relu',strides = 2,padding = 'same'))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(64,activation = 'relu'))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy',optimizer = 'rmsprop',metrics = ['accuracy'])
    model.summary()
    return model
_,m,n,o = x_train.shape

In [ ]:
#K-fold validation training 
#Choose a value for k 
k = 5
nValSamples = len(x_train)//k
nEpochs = 50
accuracy_scores = []; lossKfold = []
for i in range(k):
    print("Processing fold #",i)
    valData = x_train[i*nValSamples:(i+1)*nValSamples]
    valTargets = Y2[i*nValSamples:(i+1)*nValSamples]
    trainingData = np.concatenate(
                    [x_train[:i*nValSamples],x_train[(i+1)*nValSamples:]],axis = 0)
    trainingLabels = np.concatenate(
                    [Y2[:i*nValSamples],Y2[(i+1)*nValSamples:]],axis = 0)
    model = getModel((m,n,o))
    history = model.fit(trainingData,trainingLabels,epochs = nEpochs,batch_size = 16,verbose = 0,
                       validation_data = (valData,valTargets))
    accuracy_scores.append(history.history['accuracy'])
    lossKfold.append(history.history['loss'])

The third way to train is by employing a Unet, which is used for image segmentation and quite popular in medical
imaging and deep learning. 

In [ ]:
from sklearn.model_selection import train_test_split
model = uk.getModel((m,n,o))
model.summary()
X_train,X_test,Y_train,Y_test = train_test_split(x_train,Y1,test_size = 0.2,random_state = 42)
history = model.fit(X_train,Y_train,batch_size = 16,epochs = nEpochs,validation_data = (X_test,Y_test))
uNetLoss = history.history['loss']